In [1]:
%pip install sklearn datasets


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [2]:
%pip install scikit-learn datasets


   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.9 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.9 MB 1.4 MB/s eta 0:00:07
   --- ------------------------------------ 0.8/8.9 MB 1.5 MB/s eta 0:00:06
   ----- ---------------------------------- 1.3/8.9 MB 1.9 MB/s eta 0:00:05
   ----- ---------------------------------- 1.3/8.9 MB 1.9 MB/s eta 0:00:05
   ------- -------------------------------- 1.6/8.9 MB 1.4 MB/s eta 0:00:06
   --------- ------------------------------ 2.1/8.9 MB 1.4 MB/s eta 0:00:05
   ---------- ----------------------------- 2.4/8.9 MB 1.4 MB/s eta 0:00:05
   ----------- ---------------------------- 2.6/8.9 MB 1.4 MB/s eta 0:00:05
   ------------ --------------------------- 2.9/8.9 MB 1.4 MB/s eta 0:00:05
   -------------- ------------------------- 3.1/8.9 MB 1.5 MB/s eta 0:00:04
   --------------- --------------

In [3]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset    

c:\Users\Vansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data_path = "../data/jutsus.jsonl"
df = pd.read_json(data_path, lines=True)
df.head()

,jutsu_name,jutsu_type,jutsu_description
0,Adamantine Sealing Chains: Spiral Formation,"Hiden, Ninjutsu, Fūinjutsu, Barrier Ninjutsu, ...",Kushina uses her chains to form a barrier whil...
1,Adamantine Power: Acala,"Kekkei Genkai, Ninjutsu, Taijutsu",Hashirama kicks the opponent away and raises s...
2,Adamantine Prison Wall,"Ninjutsu, Clone Techniques, Bukijutsu","After using Transformation: Adamantine Staff, ..."
3,Adamantine Seal: Monkey Yang Suppression,"Ninjutsu, Fūinjutsu, Cooperation Ninjutsu","After placing fūinjutsu tags in an area, the u..."
4,Acrobat,"Taijutsu, Kenjutsu","The Acrobat (荒繰鷺伐刀, Akurobatto) is a kenjutsu ..."


In [5]:
def simplify_jutsu(jutsu):
    if "Genjutsu" in jutsu:
        return "Genjutsu"
    if "Ninjutsu" in jutsu:
        return "Ninjutsu"
    if "Taijutsu" in jutsu:
        return "Taijutsu"
df['jutsu_type_simplified'] = df['jutsu_type'].apply(simplify_jutsu)
df.head()


,jutsu_name,jutsu_type,jutsu_description,jutsu_type_simplified
0,Adamantine Sealing Chains: Spiral Formation,"Hiden, Ninjutsu, Fūinjutsu, Barrier Ninjutsu, ...",Kushina uses her chains to form a barrier whil...,Ninjutsu
1,Adamantine Power: Acala,"Kekkei Genkai, Ninjutsu, Taijutsu",Hashirama kicks the opponent away and raises s...,Ninjutsu
2,Adamantine Prison Wall,"Ninjutsu, Clone Techniques, Bukijutsu","After using Transformation: Adamantine Staff, ...",Ninjutsu
3,Adamantine Seal: Monkey Yang Suppression,"Ninjutsu, Fūinjutsu, Cooperation Ninjutsu","After placing fūinjutsu tags in an area, the u...",Ninjutsu
4,Acrobat,"Taijutsu, Kenjutsu","The Acrobat (荒繰鷺伐刀, Akurobatto) is a kenjutsu ...",Taijutsu


In [6]:
df['jutsu_type_simplified'].value_counts()

jutsu_type_simplified
Ninjutsu    2255
Taijutsu     397
Genjutsu     101
Name: count, dtype: int64

In [12]:
from bs4 import BeautifulSoup
class Cleaner():
    def __init__(self):
        pass 
    
    def put_line_breaks(self, text):
        return text.replace("<\p>", "<\p>\n")
    
    def remove_html_tags(self, text):
        clean_text = BeautifulSoup(text, "lxml").text
        return clean_text

    def clean(self, text):
        text = self.put_line_breaks(text)
        text = self.remove_html_tags(text)
        text = text.strip()
        return text

In [13]:
text_column_name = 'text'
label_column_name = "jutsus"
# Clean Text
cleaner = Cleaner()
df['text_cleaned'] = df[text_column_name].apply(cleaner.clean)

In [14]:
df.head(2)

,text,jutsus,text_cleaned
0,Adamantine Sealing Chains: Spiral Formation. K...,Ninjutsu,Adamantine Sealing Chains: Spiral Formation. K...
1,Adamantine Power: Acala. Hashirama kicks the o...,Ninjutsu,Adamantine Power: Acala. Hashirama kicks the o...


In [15]:
# Encode Labels 
le = preprocessing.LabelEncoder()
le.fit(df[label_column_name].tolist())

LabelEncoder()

In [16]:
label_dict = {index:label_name for index, label_name in enumerate(le.__dict__['classes_'].tolist())}
label_dict

{0: 'Genjutsu', 1: 'Ninjutsu', 2: 'Taijutsu'}

In [17]:
df['label'] = le.transform(df[label_column_name].tolist())
df.head()

,text,jutsus,text_cleaned,label
0,Adamantine Sealing Chains: Spiral Formation. K...,Ninjutsu,Adamantine Sealing Chains: Spiral Formation. K...,1
1,Adamantine Power: Acala. Hashirama kicks the o...,Ninjutsu,Adamantine Power: Acala. Hashirama kicks the o...,1
2,Adamantine Prison Wall. After using Transforma...,Ninjutsu,Adamantine Prison Wall. After using Transforma...,1
3,Adamantine Seal: Monkey Yang Suppression. Afte...,Ninjutsu,Adamantine Seal: Monkey Yang Suppression. Afte...,1
4,"Acrobat. The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...",Taijutsu,"Acrobat. The Acrobat (荒繰鷺伐刀, Akurobatto) is a ...",2


In [18]:
test_size = 0.2
df_train, df_test = train_test_split(df, 
                                     test_size=test_size, 
                                     stratify=df['label'],)
df_train['jutsus'].value_counts()

jutsus
Ninjutsu    1804
Taijutsu     317
Genjutsu      81
Name: count, dtype: int64

In [19]:
model_name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

c:\Users\Vansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vansh\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Vansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\transf

In [20]:
def preprocess_function(tokenizer,examples):
    return tokenizer(examples['text_cleaned'],truncation=True)
# Conver Pandas to a hugging face dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

# tokenize the dataset
tokenized_train = train_dataset.map(lambda examples: preprocess_function(tokenizer, examples),
                                    batched=True)
tokenized_test = test_dataset.map(lambda examples: preprocess_function(tokenizer, examples),
                                    batched=True)

Map: 100%|██████████| 551/551 [00:00<00:00, 10062.14 examples/s]
